Following the steps from https://developers.google.com/gmail/api/quickstart/python?authuser=1

**Step 1**: Enable the gmail api and save the client configuration file.

**Step 2**: Install google client library

In [14]:
# !pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [10]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from apiclient import errors
import base64
from pathlib import Path

# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

def get_service():
    """Shows basic usage of the Gmail API.
    Lists the user's Gmail labels.
    """
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('gmail_chinta512_credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('gmail', 'v1', credentials=creds)
    return service

# copied from https://developers.google.com/gmail/api/v1/reference/users/messages/list#examples
def ListMessagesMatchingQuery(service, user_id, query=''):
    """List all Messages of the user's mailbox matching the query.

    Args:
    service: Authorized Gmail API service instance.
    user_id: User's email address. The special value "me"
    can be used to indicate the authenticated user.
    query: String used to filter messages returned.
    Eg.- 'from:user@some_domain.com' for Messages from a particular sender.

    Returns:
    List of Messages that match the criteria of the query. Note that the
    returned list contains Message IDs, you must use get with the
    appropriate ID to get the details of a Message.
    """
    try:
        response = service.users().messages().list(userId=user_id,
                                                   q=query).execute()
        messages = []
        if 'messages' in response:
            messages.extend(response['messages'])

        while 'nextPageToken' in response:
            page_token = response['nextPageToken']
            response = service.users().messages().list(userId=user_id, q=query,
                                             pageToken=page_token).execute()
            messages.extend(response['messages'])

        return messages
    except errors.HttpError as e:
        print('An error occurred: %s' % e)
            
def get_attachments(service, user_id, message_id, save_dir='./attachments', overwrite=False):
    message = service.users().messages().get(userId=user_id, id=message_id).execute()
    for part in message['payload']['parts']:
        if part['filename']:
            att_id = part['body']['attachmentId']
            path = Path.cwd()/save_dir/part['filename']
            
            #do not download again if file already present
            if not overwrite and Path.exists(path):
                continue
                
            #download the attachment
            att = service.users().messages().attachments().get(userId=user_id, messageId=message_id,id=att_id).execute()
            data = att['data']
            file_data = base64.urlsafe_b64decode(data.encode('UTF-8'))
            
            Path.mkdir(Path.cwd()/save_dir, exist_ok=True)
            with open(path, 'wb') as f:
                f.write(file_data)
            print(f'wrote attachment to {path.as_posix()}')

#### Get list of message ids

In [7]:
service = get_service()
query = "from:whispers@sahajmarg.info has:attachment"
user_id = "me"
messages = ListMessagesMatchingQuery(service, user_id, query)

In [8]:
len(messages)

3312

#### Fetch attachments

In [13]:
for idx, message in enumerate(messages):
    get_attachments(service, user_id, message['id']) 